In [ ]:
import robotic as ry
import time
import numpy as np
import os
import time
import matplotlib.pyplot as plt
C = ry.Config()
C.addFile("rai-robotModels/scenarios/pandasFight.g")

C.setJointState(np.array([ 0.  , -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.  ,  0.  ,
       -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.]))
q0 = C.getJointState()
C.view()

In [17]:
armed= np.load('armed.npy')
# C.setJointState(path[0,:])
S = ry.Simulation(C, ry.SimulationEngine.physx, verbose=0)

In [18]:
def simulationCloseGrippers(S, tau=0.01):
    S.closeGripper('l_gripper', width=0.0001, speed=0.5)
    while (not S.getGripperIsGrasping('l_gripper')) and (S.getGripperWidth('l_gripper') > 0.001):
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()
    
    S.closeGripper('r_gripper', width=0.0001, speed=0.5)
    while (not S.getGripperIsGrasping('r_gripper')) and (S.getGripperWidth('r_gripper') > 0.001):
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()


def simulationOpenGrippers(S, tau=0.01):
    S.openGripper('l_gripper', width=0.05, speed=0.5)
    while S.getGripperWidth('l_gripper') < 0.05 - 0.01:
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()
    S.openGripper('r_gripper', width=0.05, speed=0.5)
    while S.getGripperWidth('r_gripper') < 0.05 - 0.01:
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()

def simulationGoTo(S, q, tau=0.01, checkCol=False):
    checkColTime = 0.1
    timer = 0
    while np.linalg.norm(S.get_q() - q) > 0.01:
        time.sleep(tau)
        S.step(q, tau, ry.ControlMode.position)
        timer += tau
        #C.view()
        if timer > 4:
            print("Target cannot be reached within 4 seconds.")
            break
        if checkCol:
            if timer > checkColTime:
                if findCollision(C, 'sword_1'):
                    return
                else:
                    checkColTime += 0.1

def simulationGoHome(S,tau=0.01):
    checkColTime = 0.1
    timer = 0
    while np.linalg.norm(S.get_q() - q0) > 0.01:
        time.sleep(tau)
        S.step(q0, tau, ry.ControlMode.position)
        #C.view()
        timer += tau
        # if timer > checkColTime:
        #     if findCollision(C, 'sword_1'):
        #         break
        #     else:
        #         checkColTime += 0.1
            
def simulationFollowPath(S, path, tau=0.01):
    checkColTime = 0.1
    timer = 0
    for i in range(len(path)):
        while np.linalg.norm(S.get_q() - path[i]) > 0.3:
            time.sleep(tau)
            print(i)
            print(np.linalg.norm(S.get_q() - path[i]))
            S.step(path[i], tau, ry.ControlMode.position)
            #C.view()
            timer += tau
            if timer > checkColTime:
                print('check_col')
                if findCollision(C, 'sword_1'):
                    return
                else:
                    checkColTime += 0.1

    simulationGoTo(S,C, path[-1], checkCol=True)

def simulationWait(S, t, tau=0.01):
    for k in range(int(t / tau)):
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()

def simulationTorqueCtrl(S, t, torque, tau=0.01):
    for k in range(int(t / tau)):
        time.sleep(tau)
        S.step(torque, tau, ry.ControlMode.acceleration)
        #C.view()

def simulationVelocityCtrl(S,t,vel,tau=0.01):
    for k in range(int(t/tau)):
        time.sleep(tau)
        S.step(vel,tau,ry.ControlMode.velocity)
        #C.view()

def followInterpolatedPath(S,path,tau=0.01):
    checkColTime = 0.1
    timer = 0
    for i in range(len(path)):
        time.sleep(tau)
        print(i)
        print(np.linalg.norm(S.get_q()- path[i]))
        S.step(path[i],tau,ry.ControlMode.position)
        #C.view()
        timer += tau
        if timer > checkColTime:
            if findCollision(C,'sword_1'):
                return
            else:
                checkColTime += 0.1
    simulationGoTo(S,path[-1],checkCol=True)

def findCollision(C,object1):
    collisions = [col for col in C.getCollisions(0) if object1 in col and not col[1].startswith('l_') and not col[0].startswith('l_')]
    if len(collisions) > 0:
        return True
    else:
        return False
    
def findCollisionshield(C):
    collisions = [col for col in C.getCollisions(0) if 'shi' in col and 'sword_1' in col]
    if len(collisions) > 0:
        return True
    else:
        return False
    
def findCollisionsword(C):
    collisions = [col for col in C.getCollisions(0) if 'sword_1' in col ]
    if len(collisions) > 0:
        return True
    else:
        return False
# Use this code. It is awesome. It will follow the path and stop if there is a collision.    
def followSplinePath(S,path,t,tau=0.01):
    """
        Guides a simulation object along a specified spline path.
        Parameters:
        S (Simulation): The simulation object that will follow the path.
        path (array): A list of waypoints defining the spline path.
        t (float): The total time duration for following the path.
        tau (float, optional): The time step for each simulation step. Default is 0.01.
        The function performs the following steps:
        1. Resets the spline reference of the simulation object.
        2. Sets the spline reference with the given path and a time vector.
        3. Iteratively steps through the simulation, updating the view and checking for collisions.
        4. If a collision with 'sword_1' is detected, the function exits early.
        5. The loop runs for a duration slightly longer than `t` to ensure completion.
        Note:
        - The `+20` in the loop range is a heuristic to prevent early termination when `t` is low.
        - The function checks for collisions every 0.1 seconds.

    """
    checkColTime = 0.1
    timer = 0
    S.resetSplineRef() # Reset previous spline reference
    S.setSplineRef(path,np.linspace(0.01,t,len(path))) # Set new spline reference
    joint_data = np.empty((int(t/tau)+20, 2,S.get_q().shape[0])) # Initialize array to store joint data
    for k in range(int(t/tau)+20): # This +20 is heuristic. It was stopping after a short time when t was low.
        time.sleep(tau)
        S.step([],tau,ry.ControlMode.spline)
        inst_pos = S.get_q().copy()
        inst_vel = S.get_qDot().copy()
        C.view()
        timer += tau
        joint_data[k,0,:] = inst_pos
        joint_data[k,1,:] = inst_vel
        if timer > checkColTime: # Check for collisions every 0.1 seconds
            if findCollision(C,'sword_1'): # Check if sword is colliding with something (left robot is excluded (no self collision))
                return joint_data[:k,:,:] # Return the joint data until collision
            else:
                checkColTime += 0.1 # Increment the collision check time

    return joint_data
    #simulationGoTo(S,path[-1],checkCol=True) 

def initializeSimulation(init_pos,testing=False):

    S = ry.Simulation(C, ry.SimulationEngine.physx, verbose=0)

    simulationGoTo(S,armed)
    simulationCloseGrippers(S)
    simulationGoTo(S,init_pos)

    shield = C.getFrame('shi')
    robot_base = C.getFrame('r_panda_base')
    sword = C.getFrame('sword_1')
    if testing:
        C.view()
    return S,shield,robot_base,sword
armed = np.load('armed.npy')

In [19]:
def defence_path(robot_pos,sword_pos,shield_pos,hitting_point,C):

    target_pos = np.array([(sword_pos[0]*0.65+hitting_point[0]*0.35),(sword_pos[1]*0.65+hitting_point[1]*0.35),(sword_pos[2]*0.65+hitting_point[2]*0.35)])

    target = C.addFrame('target', 'world')
    target.setShape(ry.ST.marker, [0.1])
    target.setPosition(target_pos)
    shield_allign = shield_pos - sword_pos
    C.setJointState(robot_pos)
    
    komo = ry.KOMO(C, 1, 10, 2, True)
    komo.addControlObjective([], 2, 1e1)
    komo.addControlObjective([], 0, 1e1)
    komo.addObjective([], ry.FS.vectorZ, ['shi'], ry.OT.eq, [1e1], shield_allign)
    #komo.addObjective([1.], ry.FS.vectorZ, [shield_allign], ry.OT.eq, [1e2], [0,0,-1])
    komo.addObjective([0.,1.], ry.FS.positionDiff, ['shi', 'target'], ry.OT.eq, [1e2])
    # komo.addObjective([0.,0.95], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])

    ret = ry.NLP_Solver() \
        .setProblem(komo.nlp()) \
        .setOptions( stopTolerance=1e-2) \
        .solve()

    path = komo.getPath()
    C.delFrame('target')
    return path[8]

In [20]:
import scipy
def interpolate_path(path,N):
    cubic_spline_interpolator = scipy.interpolate.CubicSpline(np.arange(path.shape[0]), path, axis = 0) 
    interp_path = cubic_spline_interpolator(np.linspace(0,path.shape[0]-1,N))

    return interp_path

In [ ]:
import os 
tau = 0.01
label_true = True
train_data = np.empty((0,28))
train_label = np.empty((0,7))

step_counter = 0
for file in os.listdir('real_spline_attackPaths'):
        C = ry.Config()
        C.addFile("rai-robotModels/scenarios/pandasFight.g")
        q0 = C.getJointState()
        C.setJointState(np.array([ 0.  , -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.  ,  0.  ,
        -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.]))
        attack_path = np.load('real_spline_attackPaths/'+file)
        len_attack = attack_path.shape[0]
        if len_attack > 10:
            step_counter +=1
            hiting_point = np.load('attackPaths/hitting_point_'+file[-5:])
            init_pos = attack_path[0,0,:]
            S,shield,robot_base,sword = initializeSimulation(init_pos)
            k = 0
            defence_data = np.empty((attack_path.shape[0],2,14))

            for row_no in range(1,int(np.floor(attack_path.shape[0]/3))):

                if row_no %3 == 1:
                    def_target_pos = defence_path(init_pos,sword.getPosition(),shield.getPosition(),hiting_point,C)
                    label_true = True
                
                row = attack_path[row_no*3,0,:]
                attacker = row[:7]
                defender = def_target_pos[7:]    

                pos = S.get_q() / np.pi
                vel = S.get_qDot() / 10
                
                q = np.concatenate([attacker,defender]) 
                S.step(q, tau, ry.ControlMode.position)

                if label_true == True:
                    train_input = np.concatenate([pos,vel])[np.newaxis,...]
                    train_data = np.concatenate([train_data,train_input],axis=0)
                    train_label = np.append(train_label,defender[np.newaxis,...],axis=0)
                label_true = False
                k += 1
                init_pos = q
                if step_counter % 10 == 0:
                    np.save(f'defencebot_imitation_traininput{step_counter}.npy',train_data)
                    np.save(f'defencebot_imitation_trainlabel{step_counter}.npy',train_label)
                if findCollisionshield(C) == True:
                    break
                if findCollision(C,"sword_1") == True:
                    break

            # defence_data = defence_data[:k,:,7:]
            # if k > 5:  
            #     defence_data = interpolate_path(defence_data,len_attack)   
            #     total_path = np.concatenate([attack_path[:,:,:7],defence_data],axis=2)
            #     np.save('defence paths/'+file,total_path)    
            del S
            del C
np.save('defencebot_imitation_traininput.npy',train_data)
np.save('defencebot_imitation_trainlabel.npy',train_label)

In [ ]:
import robotic as ry
import time
import numpy as np
import os
import time
import matplotlib.pyplot as plt
tau = 0.01


C = ry.Config()
C.addFile("rai-robotModels/scenarios/pandasFight.g")
q0 = C.getJointState()
C.setJointState(np.array([ 0.  , -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.  ,  0.  ,
-1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.]))

fight_path = np.load("defence paths/path_321.npy")
print(fight_path.shape)
init_pos = fight_path[0,0,:]
S,shield,robot_base,sword = initializeSimulation(init_pos,testing=True)
time.sleep(3)

k = 0
fight_len = fight_path.shape[0]
k = 0
attack_means = []
defence_means = []
for row in fight_path:
    S.step(row[0,:], tau, ry.ControlMode.position)
    vel = S.get_qDot()
    attack_means.append(np.mean(np.abs(vel[:7])))
    defence_means.append(np.mean(np.abs(vel[7:])))
    C.view()
    time.sleep(0.05)
    k+=1
print(f"attack mean: {np.mean(attack_means)}")
print(f"defence mean: {np.mean(defence_means)}")
print(f'defence max: {np.max(np.abs(fight_path[:,:,:7]))}')
print(f'attack max: {np.max(np.abs(fight_path[:,:,7:]))}')

        